# Descarga del Cuadro 1 desde SPensiones

En esta clase vamos a replicar, de forma simplificada, la lógica utilizada en el proyecto **FondosdePensiones** para descargar información desde la Superintendencia de Pensiones.

El objetivo será:

1. Acceder al índice mensual de carteras agregadas.
2. Obtener el primer cuadro disponible en formato HTML.
3. Descargar el contenido del cuadro.
4. Decodificar correctamente el HTML.
5. Transformar los números desde formato chileno a formato estándar.
6. Convertir la tabla en un DataFrame utilizando pandas.

Este será el insumo para construir posteriormente un panel de datos mensual.

In [1]:
#LIBRERÍAS
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from io import StringIO

## Problemas comunes con datos de SPensiones

Al descargar tablas desde SPensiones nos enfrentamos a:

### 🔤 Problemas de codificación

Ejemplo:

INVERSIÓN → INVERSIÃN  
Esto ocurre porque el HTML no viene en UTF-8.

---

### 🔢 Formato numérico europeo

Las tablas contienen números como:

| Original | Significado |
|----------|-------------|
| 4.471,70 | 4471.70 |
| 12.345 | 12345 |
| 123,4 | 123.4 |

Este formato no es reconocido por pandas, por lo que debemos transformarlo previamente.

In [2]:
#DECODIFICACIÓN (DEL PROYECTO)
def decode_html(resp):

    raw = resp.content

    try:
        return raw.decode("utf-8")
    except UnicodeDecodeError:
        pass

    enc = (resp.apparent_encoding or "").lower().strip()
    if enc:
        try:
            return raw.decode(enc, errors="replace")
        except Exception:
            pass

    return raw.decode("latin1", errors="replace")

## Transformación del formato numérico

Antes de leer la tabla con pandas debemos transformar:

1.234,56 → 1234.56  
12.345 → 12345  
123,4 → 123.4  

⚠️ IMPORTANTE:

Debemos modificar SOLO los números dentro del HTML,
sin alterar texto, encabezados ni atributos.

In [3]:
#TRANSFORMACIÓN NUMÉRICA (DEL PROYECTO)
_RE_CH_NUM = re.compile(
    r"\b\d{1,3}(?:\.\d{3})*(?:,\d+)?\b|\b\d+(?:,\d+)\b|\b\d+\b"
)

def _to_float_token(token):
    return token.replace(".", "").replace(",", ".")

def _transformar_solo_numeros_en_texto(text):

    def _repl(m):
        return _to_float_token(m.group(0))

    return _RE_CH_NUM.sub(_repl, text)

def _html_transformar_solo_numeros(html):

    soup = BeautifulSoup(html, "html.parser")

    for node in soup.find_all(string=True):

        if node.parent.name in ("script", "style"):
            continue

        original = str(node)

        if not original.strip():
            continue

        nuevo = _transformar_solo_numeros_en_texto(original)

        if nuevo != original:
            node.replace_with(nuevo)

    return str(soup)

## Descarga del Cuadro 1

SPensiones publica múltiples cuadros mensuales.

Para simplificar:

Tomaremos el **primer link disponible en formato HTML**
correspondiente al Cuadro 1.

## 1. Creación de la sesión HTTP

Para acceder a SPensiones debemos simular un navegador web.

Esto se realiza mediante el objeto `Session()` de la librería `requests`.

Esto permite:

- Mantener conexión persistente (Keep-Alive)
- Reducir latencia
- Enviar encabezados HTTP personalizados

En particular debemos indicar:

- `User-Agent`: identifica el cliente como navegador
- `Referer`: página de origen requerida por SPensiones

In [4]:
s = requests.Session()

s.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php"
})

## 2. Acceso al índice mensual

SPensiones organiza los cuadros mensuales mediante
una página intermedia que actúa como índice.

La URL contiene el parámetro:

- `periodo`: formato YYYYMM

In [5]:
BASE = "https://www.spensiones.cl"
periodo = "202001"

url = f"{BASE}/apps/loadCarteras/loadCarAgr.php?menu=sci&menuN1=estfinfp&menuN2=NOID&orden=20&periodo={periodo}&ext=.php"

r = s.get(url)

## 3. Extracción del link del Cuadro

La página anterior no contiene directamente la tabla.

Contiene enlaces que apuntan a los cuadros en HTML.

Utilizamos BeautifulSoup para:

- Parsear el HTML
- Identificar los enlaces disponibles
- Tomar el primer link correspondiente al Cuadro 1

In [6]:
soup = BeautifulSoup(r.text, "html.parser")

a = soup.find("a", title="Html")

if a is None:
    print("No hay cuadro disponible")

real_link = urljoin(BASE, a["href"])
print(real_link)

https://www.spensiones.cl/apps/carteras/genera_xsl_v2.0.php?param=L09hMXNCSkZ6Qk5yQ0hKK1phTmdjZjVGSklOOS9tcWJxVUFyelc4MjhwMCtrWWU0dlk3MVRnPT0=


## 4. Descarga del HTML del Cuadro

Ahora descargamos el contenido real de la tabla.

⚠️ IMPORTANTE:

No utilizamos `.text`
ya que puede generar problemas de codificación.

In [7]:
r2 = s.get(real_link)
html = decode_html(r2)

## 5. Conversión del formato numérico

Los datos se encuentran en formato europeo:

1.234,56 → 1234.56

Debemos transformar únicamente los números
sin modificar texto o encabezados.

In [8]:
html_transformado = _html_transformar_solo_numeros(html)

In [9]:
print(html_transformado)

<html>
<head>
<title>Superintendencia de Pensiones - SP</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="https://www.spensiones.cl/css/appscreen.css" rel="stylesheet" type="text/css"/>
<script src="/javascript/functions.js" type="text/javascript"></script>
</head>
<body><div class="col-sm-10 col-sm-offset-1">
<h3>CUADRO Nº 1: CARTERA AGREGADA DE LOS FONDOS DE PENSIONES POR TIPO DE FONDO</h3>
<h3>Al 31-01-2020 - En millones de dólares y porcentaje</h3>
<table class="table">
<tr>
<th align="center" rowspan="2"> </th>
<th align="center" colspan="2">A</th>
<th align="center" colspan="2">B</th>
<th align="center" colspan="2">C</th>
<th align="center" colspan="2">D</th>
<th align="center" colspan="2">E</th>
<th align="center" colspan="2">TOTAL</th>
</tr>
<tr>
<th>MMUS$</th>
<th>%Fondo</th>
<th>MMUS$</th>
<th>%Fondo</th>
<th>MMUS$</th>
<th>%Fondo</th>
<th>MMUS$</th>
<th>%Fondo</th>
<th>MMUS$</th>
<th>%Fondo</th>
<th>MMUS$</th>
<th>%Fondo</th>
</tr>
<tr>

## 6. Conversión a DataFrame

Finalmente utilizamos pandas para:

- Leer la tabla HTML
- Detectar encabezado multinivel
- Convertir columnas numéricas

In [10]:
from io import StringIO

tablas = pd.read_html(StringIO(html_transformado), header=[0,1])

df = tablas[0]

for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.head()

Unnamed: 0_level_0        A                B                C  \
                Unnamed: 0_level_1    MMUS$ %Fondo     MMUS$ %Fondo     MMUS$   
0         INVERSIÓN NACIONAL TOTAL  4471.70  16.09  10903.82  33.05  37625.73   
1                   RENTA VARIABLE  3802.30  13.68   3821.15  11.58   6239.95   
2                         Acciones  3123.52  11.24   3281.80   9.95   5206.25   
3  Fondos de Inversión y Otros (4)   621.12   2.23    470.21   1.43    871.06   
4         Activos Alternativos (5)    57.67   0.21     69.15   0.21    162.64   

                 D                E             TOTAL         
  %Fondo     MMUS$ %Fondo     MMUS$ %Fondo      MMUS$ %Fondo  
0  50.99  25176.07  70.19  32144.42  90.90  110321.73  53.60  
1   8.46   1359.31   3.79    728.47   2.06   15951.18   7.75  
2   7.06   1104.92   3.08    722.18   2.04   13438.68   6.53  
3   1.18    213.46   0.60      4.05   0.01    2179.89   1.06  
4   0.22     40.93   0.11      2.23   0.01     332.61   0.16

## Automatización del proceso

Una vez comprendidas las etapas del proceso,
podemos encapsularlas en una función
para facilitar su reutilización.

In [11]:
#FUNCIÓN FINAL
def bajar_primer_cuadro(periodo):

    BASE = "https://www.spensiones.cl"

    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php"
    })

    url = f"{BASE}/apps/loadCarteras/loadCarAgr.php?menu=sci&menuN1=estfinfp&menuN2=NOID&orden=20&periodo={periodo}&ext=.php"

    r = s.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    a = soup.find("a", title="Html")

    if a is None:
        print("❌ No hay link")
        return None

    real_link = urljoin(BASE, a["href"])

    r2 = s.get(real_link)
    html = decode_html(r2)

    html_transformado = _html_transformar_solo_numeros(html)

    tablas = pd.read_html(StringIO(html_transformado), header=[0,1])

    df = tablas[0]

    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

## Ejemplo: Periodo Enero 2020

Ahora descargaremos el Cuadro 1 correspondiente al periodo 202001.

In [12]:
#EJECUCIÓN
df = bajar_primer_cuadro("202001")
df.head()

Unnamed: 0_level_0        A                B                C  \
                Unnamed: 0_level_1    MMUS$ %Fondo     MMUS$ %Fondo     MMUS$   
0         INVERSIÓN NACIONAL TOTAL  4471.70  16.09  10903.82  33.05  37625.73   
1                   RENTA VARIABLE  3802.30  13.68   3821.15  11.58   6239.95   
2                         Acciones  3123.52  11.24   3281.80   9.95   5206.25   
3  Fondos de Inversión y Otros (4)   621.12   2.23    470.21   1.43    871.06   
4         Activos Alternativos (5)    57.67   0.21     69.15   0.21    162.64   

                 D                E             TOTAL         
  %Fondo     MMUS$ %Fondo     MMUS$ %Fondo      MMUS$ %Fondo  
0  50.99  25176.07  70.19  32144.42  90.90  110321.73  53.60  
1   8.46   1359.31   3.79    728.47   2.06   15951.18   7.75  
2   7.06   1104.92   3.08    722.18   2.04   13438.68   6.53  
3   1.18    213.46   0.60      4.05   0.01    2179.89   1.06  
4   0.22     40.93   0.11      2.23   0.01     332.61   0.16

## Guardado a CSV (Opcional)

Para utilizar esta información posteriormente
(en Excel o para construir paneles de datos),
podemos guardar el DataFrame como CSV.

In [13]:
#GUARDAR
df.to_csv("cartera_agregada_202001.csv", index=False, encoding="utf-8-sig")

# Descarga masiva: todos los meses de un año

Hasta ahora hemos descargado el Cuadro 1 para un único periodo.

Sin embargo, en análisis financiero y econométrico
necesitamos construir **series de tiempo** o **paneles de datos**.

Para esto debemos descargar la información de múltiples meses.

---

## Objetivo

Descargar el Cuadro 1 para:

202001  
202002  
...  
202012  

Es decir, todos los meses del año 2020.

## Generación automática de periodos

Los periodos deben estar en formato:

YYYYMM

Por ejemplo:

202001  
202002  
202003  

Podemos generar estos valores utilizando un ciclo `for`.

In [14]:
periodos = []

for mes in range(1,13):
    periodo = f"2020{mes:02d}"
    periodos.append(periodo)

periodos

['202001',
 '202002',
 '202003',
 '202004',
 '202005',
 '202006',
 '202007',
 '202008',
 '202009',
 '202010',
 '202011',
 '202012']

## Descarga iterativa de los cuadros

Ahora utilizaremos:

- Nuestra función `bajar_primer_cuadro()`
- Un ciclo `for`
- Una lista para almacenar los resultados

Cada tabla será guardada como un DataFrame independiente.

In [15]:
dataframes = []

for periodo in periodos:

    print(f"Procesando {periodo}")

    try:
        df = bajar_primer_cuadro(periodo)

        if df is not None:

            # 🔥 agregar columna PERIODO
            df["PERIODO"] = periodo

            dataframes.append(df)

    except Exception as e:
        print(f"Error en {periodo}: {e}")

Procesando 202001
Procesando 202002
Procesando 202003
Procesando 202004
Procesando 202005
Procesando 202006
Procesando 202007
Procesando 202008
Procesando 202009
Procesando 202010
Procesando 202011
Procesando 202012


## Verificación de resultados

Podemos verificar cuántos meses fueron descargados correctamente.

In [16]:
len(dataframes)

12

## Construcción de panel de datos

Una vez descargadas todas las tablas mensuales,
podemos combinarlas en un único DataFrame
utilizando `pd.concat()`.

In [17]:
panel_2020 = pd.concat(dataframes, ignore_index=True)

panel_2020

Unnamed: 0_level_0         A                 B          \
                  Unnamed: 0_level_1     MMUS$  %Fondo     MMUS$  %Fondo   
0           INVERSIÓN NACIONAL TOTAL   4471.70   16.09  10903.82   33.05   
1                     RENTA VARIABLE   3802.30   13.68   3821.15   11.58   
2                           Acciones   3123.52   11.24   3281.80    9.95   
3    Fondos de Inversión y Otros (4)    621.12    2.23    470.21    1.43   
4           Activos Alternativos (5)     57.67    0.21     69.15    0.21   
..                               ...       ...     ...       ...     ...   
379                    TOTAL ACTIVOS  27292.67  100.00  29182.36  100.00   
380          SUBTOTAL RENTA VARIABLE  21842.44   80.03  17738.43   60.78   
381              SUBTOTAL RENTA FIJA   5111.13   18.73  10966.00   37.58   
382               SUBTOTAL DERIVADOS    169.76    0.62    351.41    1.20   
383                   SUBTOTAL OTROS    169.33    0.62    126.52    0.43   

            C                 D                 E              TOTAL          \
        MMUS$  %Fondo     MMUS$  %Fondo     MMUS$  %Fondo      MMUS$  %Fondo   
0    37625.73   50.99  25176.07   70.19  32144.42   90.90  110321.73   53.60   
1     6239.95    8.46   1359.31    3.79    728.47    2.06   15951.18    7.75   
2     5206.25    7.06   1104.92    3.08    722.18    2.04   13438.68    6.53   
3      871.06    1.18    213.46    0.60      4.05    0.01    2179.89    1.06   
4      162.64    0.22     40.93    0.11      2.23    0.01     332.61    0.16   
..        ...     ...       ...     ...       ...     ...        ...     ...   
379  70982.81  100.00  38066.60  100.00  48152.02  100.00  213676.45  100.00   
380  28756.22   40.51   7637.44   20.06   2350.89    4.88   78325.43   36.66   
381  41196.16   58.04  29922.28   78.61  45237.52   93.95  132433.09   61.98   
382    765.01    1.08    344.30    0.90    391.83    0.81    2022.31    0.95   
383    265.41    0.37    162.57    0.43    171.79    0.36     895.62    0.42   

    PERIODO  
             
0    202001  
1    202001  
2    202001  
3    202001  
4    202001  
..      ...  
379  202012  
380  202012  
381  202012  
382  202012  
383  202012  

[384 rows x 14 columns]

Antes de transformar a formato Long,
debemos inspeccionar los nombres de columnas.

En particular, verificar si la variable PERIODO
se mantiene correctamente después de la concatenación.

Las columnas se encuentran en formato multinivel (MultiIndex).

Debemos convertir estos encabezados en nombres planos
antes de utilizar la función melt().

In [18]:
panel_2020.columns

MultiIndex([('Unnamed: 0_level_0', 'Unnamed: 0_level_1'),
            (                 'A',              'MMUS$'),
            (                 'A',             '%Fondo'),
            (                 'B',              'MMUS$'),
            (                 'B',             '%Fondo'),
            (                 'C',              'MMUS$'),
            (                 'C',             '%Fondo'),
            (                 'D',              'MMUS$'),
            (                 'D',             '%Fondo'),
            (                 'E',              'MMUS$'),
            (                 'E',             '%Fondo'),
            (             'TOTAL',              'MMUS$'),
            (             'TOTAL',             '%Fondo'),
            (           'PERIODO',                   '')],
           )

# Wide → Long (correcto) usando `stack` sobre MultiIndex

Nuestro `panel_2020` está en formato **Wide** y sus columnas son un **MultiIndex**:

- **Nivel 0**: Fondo (A, B, C, D, E, TOTAL)
- **Nivel 1**: Métrica (MMUS$, %Fondo)

En formato Long queremos que **FONDO** y **METRICA** sean variables separadas, no un string tipo `A_MMUSD`.

Por eso NO usamos `melt` juntando nombres, sino `stack(level=[0,1])`.

In [19]:
df = panel_2020.copy()
df.head()

Unnamed: 0_level_0        A                B                C  \
                Unnamed: 0_level_1    MMUS$ %Fondo     MMUS$ %Fondo     MMUS$   
0         INVERSIÓN NACIONAL TOTAL  4471.70  16.09  10903.82  33.05  37625.73   
1                   RENTA VARIABLE  3802.30  13.68   3821.15  11.58   6239.95   
2                         Acciones  3123.52  11.24   3281.80   9.95   5206.25   
3  Fondos de Inversión y Otros (4)   621.12   2.23    470.21   1.43    871.06   
4         Activos Alternativos (5)    57.67   0.21     69.15   0.21    162.64   

                 D                E             TOTAL        PERIODO  
  %Fondo     MMUS$ %Fondo     MMUS$ %Fondo      MMUS$ %Fondo          
0  50.99  25176.07  70.19  32144.42  90.90  110321.73  53.60  202001  
1   8.46   1359.31   3.79    728.47   2.06   15951.18   7.75  202001  
2   7.06   1104.92   3.08    722.18   2.04   13438.68   6.53  202001  
3   1.18    213.46   0.60      4.05   0.01    2179.89   1.06  202001  
4   0.22     40.93   0.11      2.23   0.01     332.61   0.16  202001

## 1) Identificar las columnas "id" (identificadores)

Para construir un panel Long necesitamos al menos dos identificadores:

- **TipoInstrumento**: la categoría (texto) de la fila
- **PERIODO**: el mes en formato YYYYMM

El resto de columnas corresponde a combinaciones (FONDO, METRICA).

⚠️ Detalle importante:
Cuando un DataFrame tiene columnas MultiIndex, una columna simple como PERIODO
puede quedar guardada como `('PERIODO','')` en vez de `"PERIODO"`.
Por eso primero la detectamos de forma robusta.

In [20]:
tipo_col = df.columns[0]

periodo_col = None
if "PERIODO" in df.columns:
    periodo_col = "PERIODO"
else:
    for c in df.columns:
        if isinstance(c, tuple) and str(c[0]).strip().upper() == "PERIODO":
            periodo_col = c
            break

if periodo_col is None:
    raise KeyError("No se encontró la columna PERIODO en el DataFrame")

tipo_col, periodo_col

(('Unnamed: 0_level_0', 'Unnamed: 0_level_1'), 'PERIODO')

## 2) Pasar identificadores a índice (`set_index`)

Para usar `stack`, primero convertimos nuestros identificadores en índice.
Esto significa:

- Las filas quedarán identificadas por (TipoInstrumento, PERIODO)
- Las columnas quedarán como MultiIndex (FONDO, METRICA)

En otras palabras: dejamos el DataFrame listo para “apilar” los niveles de columnas.

In [21]:
dfi = df.set_index([tipo_col, periodo_col])
dfi.head()

A                B  \
                                                    MMUS$ %Fondo     MMUS$   
(Unnamed: 0_level_0, Unnamed: 0_level_1) PERIODO                             
INVERSIÓN NACIONAL TOTAL                 202001   4471.70  16.09  10903.82   
RENTA VARIABLE                           202001   3802.30  13.68   3821.15   
Acciones                                 202001   3123.52  11.24   3281.80   
Fondos de Inversión y Otros (4)          202001    621.12   2.23    470.21   
Activos Alternativos (5)                 202001     57.67   0.21     69.15   

                                                                C         \
                                                 %Fondo     MMUS$ %Fondo   
(Unnamed: 0_level_0, Unnamed: 0_level_1) PERIODO                           
INVERSIÓN NACIONAL TOTAL                 202001   33.05  37625.73  50.99   
RENTA VARIABLE                           202001   11.58   6239.95   8.46   
Acciones                                 202001    9.95   5206.25   7.06   
Fondos de Inversión y Otros (4)          202001    1.43    871.06   1.18   
Activos Alternativos (5)                 202001    0.21    162.64   0.22   

                                                         D                E  \
                                                     MMUS$ %Fondo     MMUS$   
(Unnamed: 0_level_0, Unnamed: 0_level_1) PERIODO                              
INVERSIÓN NACIONAL TOTAL                 202001   25176.07  70.19  32144.42   
RENTA VARIABLE                           202001    1359.31   3.79    728.47   
Acciones                                 202001    1104.92   3.08    722.18   
Fondos de Inversión y Otros (4)          202001     213.46   0.60      4.05   
Activos Alternativos (5)                 202001      40.93   0.11      2.23   

                                                             TOTAL         
                                                 %Fondo      MMUS$ %Fondo  
(Unnamed: 0_level_0, Unnamed: 0_level_1) PERIODO                           
INVERSIÓN NACIONAL TOTAL                 202001   90.90  110321.73  53.60  
RENTA VARIABLE                           202001    2.06   15951.18   7.75  
Acciones                                 202001    2.04   13438.68   6.53  
Fondos de Inversión y Otros (4)          202001    0.01    2179.89   1.06  
Activos Alternativos (5)                 202001    0.01     332.61   0.16

## 3) Transformación Wide → Long usando `stack(level=[0,1])`

`stack()` toma niveles de columnas y los baja a filas.

Aquí hacemos:

- `stack(level=0)` baja el Fondo (A,B,C...)
- `stack(level=1)` baja la Métrica (MMUS$, %Fondo)

Como queremos ambos identificadores separados, usamos:

`stack(level=[0,1])`

Resultado:

- FONDO pasa a ser una columna
- METRICA pasa a ser una columna
- y queda una columna final llamada `valor`

In [22]:
panel_long = (
    dfi
    .stack(level=[0, 1])
    .reset_index()
)

panel_long.head()

,"(Unnamed: 0_level_0, Unnamed: 0_level_1)",PERIODO,level_2,level_3,0
0,INVERSIÓN NACIONAL TOTAL,202001,A,MMUS$,4471.70
1,INVERSIÓN NACIONAL TOTAL,202001,A,%Fondo,16.09
2,INVERSIÓN NACIONAL TOTAL,202001,B,MMUS$,10903.82
3,INVERSIÓN NACIONAL TOTAL,202001,B,%Fondo,33.05
4,INVERSIÓN NACIONAL TOTAL,202001,C,MMUS$,37625.73


## 4) Renombrar columnas finales

Después de `reset_index()` pandas deja nombres genéricos.

Queremos un panel limpio con columnas:

- TipoInstrumento
- PERIODO
- FONDO
- METRICA
- valor

Esto deja el dataset listo para análisis econométrico, gráficos y concatenación de periodos.

In [23]:
panel_long.columns = ["TipoInstrumento", "PERIODO", "FONDO", "METRICA", "valor"]
panel_long.head()

,TipoInstrumento,PERIODO,FONDO,METRICA,valor
0,INVERSIÓN NACIONAL TOTAL,202001,A,MMUS$,4471.70
1,INVERSIÓN NACIONAL TOTAL,202001,A,%Fondo,16.09
2,INVERSIÓN NACIONAL TOTAL,202001,B,MMUS$,10903.82
3,INVERSIÓN NACIONAL TOTAL,202001,B,%Fondo,33.05
4,INVERSIÓN NACIONAL TOTAL,202001,C,MMUS$,37625.73


## (Opcional) Normalizar métricas

Para estandarizar el panel:

- MMUS$ → MMUSD
- %Fondo → PCT

Esto evita problemas al concatenar años o hacer análisis.

In [24]:
panel_long["METRICA"] = (
    panel_long["METRICA"]
    .astype(str)
    .str.replace("MMUS$", "MMUSD", regex=False)
    .str.replace("%Fondo", "PCT", regex=False)
)

panel_long.head()

,TipoInstrumento,PERIODO,FONDO,METRICA,valor
0,INVERSIÓN NACIONAL TOTAL,202001,A,MMUSD,4471.70
1,INVERSIÓN NACIONAL TOTAL,202001,A,PCT,16.09
2,INVERSIÓN NACIONAL TOTAL,202001,B,MMUSD,10903.82
3,INVERSIÓN NACIONAL TOTAL,202001,B,PCT,33.05
4,INVERSIÓN NACIONAL TOTAL,202001,C,MMUSD,37625.73


## (Opcional) Normalizar nombres de métricas

A veces el segundo nivel trae `MMUS$` y `%Fondo`.
Podemos renombrarlo a:

- MMUS$ → MMUSD
- %Fondo → PCT

Esto deja el panel consistente para análisis y para concatenar años.

In [25]:
panel_long["METRICA"] = (
    panel_long["METRICA"]
    .astype(str)
    .str.replace("MMUS$", "MMUSD", regex=False)
    .str.replace("%Fondo", "PCT", regex=False)
)

panel_long.head()

,TipoInstrumento,PERIODO,FONDO,METRICA,valor
0,INVERSIÓN NACIONAL TOTAL,202001,A,MMUSD,4471.70
1,INVERSIÓN NACIONAL TOTAL,202001,A,PCT,16.09
2,INVERSIÓN NACIONAL TOTAL,202001,B,MMUSD,10903.82
3,INVERSIÓN NACIONAL TOTAL,202001,B,PCT,33.05
4,INVERSIÓN NACIONAL TOTAL,202001,C,MMUSD,37625.73


## Guardar el panel Long

Guardamos en CSV para usar en Excel / análisis posterior.

In [26]:
panel_long.to_csv("panel_cartera_agregada_2020_long.csv",
                   index=False, encoding="utf-8-sig")
